# Bit commitment for thrusting parties

Bit commitment is an essential and pretty simple primitive in cryptography. The protocol could be summarice as Alice 
wants to send one of two messages to Bob but She wants to forbid acces to it until She decides the correct time to discover the message. However, an important condition of the protocol is that once Alice sends her messages She can't change it.
Simple and intuitive. This protocol works like a charm in modern cryptography but it doesn't work in quantum
cryptography unless we make certain assumptions. In this case we make the assumption that both Bob and Alice
are not malintended users.

First let's define some imports and two auxiliary function to convert from bits to strings and vice versa.

In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

In the next cell, Alice chooses the messages She wants to send to Bob then She encrypts Her decision and sends it to Bob. The encryption works as follows: For each bit in the bit String, She choses a random number in the set {0,1}. If the number is 1, She encrypts the choosen bit and She leaves the bit without encryption otherwise. Notice that it would be impossible for Bob to know which bits were encrypted and which weren't.

In [2]:
#Alice cell, Bob can't see what's going in on here
m0 = tobits("Yes I want to go to the cinema with you!!!") 
m1 = tobits("No I dont want to go to the cinema with you...")
messageToSend = m1
Alice_bases = [randint(0,1) for x in range(len(messageToSend))]
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(messageToSend[i] == "1"):
            mycircuit.x(0)
    else:
        if(messageToSend[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

In the final step, Bob receives the qubits from Alice and after a while She sends the basis to him. Then Bob proceeds to decipher the original message. This consists in appliying an H gate if Alice's basis says 1. After applying these gates Bob will do some measurements and He'll be able to recover the original message.

In [3]:
#Bob cell, Alice can't see what's going in on here
backend = Aer.get_backend('qasm_simulator')
measurements = list()
for i in range(len(Alice_bases)):
    qubit = qubits[i]
    if(Alice_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))
print("Alice message was: " + frombits(measurements))

Alice message was: No I dont want to go to the cinema with you...


## Why it doesn't work if Alice cheats in the protocol?

In the first step when Alice encrypts her bits She could decide to cheat in the following way:
She makes an entangled pair of qubits with her qubits. Then if she decided to change her answer. She does a measurement, on her qubits, wich causes that Bob's qubits meassure the same as the Alice qubits. This violates the protocol assumptions that was: Alice can't change the message once She sends it.

Also, She could delay her message which contains the basis making impossible for Bob to know Alice message. So the conditions of the problem are violated once again.

The final problem comes when Alice delays the sending long enough. Due to the noisy quantum computers of today the original message would be so noisy that it would be impossible for Bob to recover the original message due to the noise in the qubits.

# References:
Broadbent, A., Schaffner, C. Quantum cryptography beyond quantum key distribution. Des. Codes Cryptogr. 78, 351–382 (2016). https://doi.org/10.1007/s10623-015-0157-4